In [ ]:
# Common imports (you may not end up using all of them)
import numpy as np
import numpy.fft as fft
import scipy.ndimage as ndi
import skimage as ski
import skimage.io as io
import skimage.exposure as ex
import matplotlib.pylab as plt
%matplotlib inline

plt.gray() # set default to grayscale
from utils import * # import all of our utilities

Blind Deconvolution
==========

Blind (or automatic) deconvolution is deconvolution of an image that was was convolved with an unknown kernel. This is understandably challenging. All methods require at least some information about the unknown kernel, such as the general form or its size. For example if we knew the PSF was an Airy disk like shown in class, we could try many different values for $\omega$ until we got one that resulted in an image that was sharp but not just noise. We would need to establish some way of measuring "sharp but not just noise".

We can also adapt the Richardson-Lucy (RL) method discussed in class to work with a known-sized kernel but with unknown values, as was shown by [Biggs and Andrews in 1997](http://www.rasmus-ischebeck.de/dr/bibliography/pdf/biggs97a.pdf). This is the method you will implement in this project.

This algorithm will be similar to the RL algorithm however both the image data and the PSF will be updated during each iteration and will require additional work to update each. **Note:** the following math will use the variables as described in the paper and will not necessarily match what we have called similar values before.

The original image and PSF are $d$ and $q$ respectively. The image and PSF results after $k$ iterations are $f_k$ and $p_k$ respectively. For iteration $k=0$, $f_0=d$ and $p_0=\frac{q}{\sum{q}}$ (so it sums to 1).

Each iteration has 3 steps: prediction, deconvolution, and update.

**Step 1:** During the predicition step we calculate the new predicted image and PSF $y_k$ and $z_k$ respectively based on the current deblurred image and PSF $f_k$ and $p_k$ along with the previous gradients for the image ($g_{k-1}$ and $g_{k-2}$) and PSF ($h_{k-1}$ and $h_{k-2}$). They are calculated as follows:
$$
\begin{align}
\alpha_k &= \frac{\sum{g_{k-1}\cdot g_{k-2}}}{\sum{g_{k-2}\cdot g_{k-2}}} &0\le\alpha_k\le1 \\
y_k &= f_k + \alpha_k \cdot (f_k - f_{k-1}) &y_k\ge0 \\
\beta_k &= \frac{\sum{h_{k-1}\cdot h_{k-2}}}{\sum{h_{k-2}\cdot h_{k-2}}} &0\le\beta_k\le1 \\
\tilde{z}_k &= p_k + \beta_k \cdot (p_k - p_{k-1}) &\tilde{z}_k\ge0 \\
z_k &= \frac{\tilde{z}_k}{\sum{\tilde{z}_k}} \\
\end{align}
$$
However, since this depends on data from the previous 2 iterations, during the first two iterations the $\alpha$ and $\beta$ values are 0 ($\alpha_0 = \alpha_1 = 0$ and $\beta_0 = \beta_1 = 0$). This means that most of these calculations do not need to be performed at all during the first 2 iterations.

**Step 2:** During the deconvolution step we perform most of an LR deconvolution using the predictions $y_k$ and $z_k$ along with the original image $d$ to obtain the relative blurred image $r_k$:
$$ r_k = \frac{d}{f_k \otimes z_k} $$

**Step 3:** During the update step we calculate the next deblurred image $f_{k+1}$ and PSF $p_{k+1}$ along with the gradients for the image $g_k$ and PSF $h_k$:
$$
\begin{align}
f_{k+1} &= y_k\cdot(r_k\otimes p_k) &f_{k+1}\ge0 \\
g_k &= f_{k+1} - y_k \\
\tilde{p}_{k+1} &= z_k\cdot(r_k\otimes f_k), &\tilde{p}_{k+1}\ge0 \\
p_{k+1} &= \frac{\tilde{p}_{k+1}}{\sum{\tilde{p}_{k+1}}} \\
h_k &= p_{k+1} - z_k
\end{align}
$$

These three steps are repeated for each iteration of the algorithm. The constraints displayed above on the variables (e.g. $0\le\alpha_k\le1$) are very important and cause the algorithm not to "explode". If a variable ever goes outside the constraints then it shold be brought to the closest legal value (e.g. is $\alpha_k$ is negative it should be brought to 0). The above math also is very careful to always make sure the value of the PSFs are always normalized so the entire PSF sums to $1$.


Python Function
========

Create the Python function `blind_deconvolution` that takes 3 arguments: the blurred image ($d$), the guessed PSF ($q$), and the number of iterations. The number of iterations should default to `10` if not provided. The provided PSF really just needs to have the right shape, the actual values in it are not nearly as important as its shape. The function returns the deconvoled image $j_k$ (but clipped to the range $0.0$ to $1.0$) and the final PSF $p_k$.

The image and PSF given to this method should be given to the `skimage.img_as_float` function to guarantee that they are floating-point images. The return values are both floating-point, real, images.

There are some variables that you will need to keep around for the next (or next two) iterations. You will have to work on managing this. But make sure not to keep all values from all iterations around. Only keep what is necessary, otherwise this is going to start taking tons of memory.

One important thing here is the convolution for $\tilde{p}_{k+1}$ is a bit different. In other convolutions we typically make the smaller of the two operands as large as the other one and just move forward. In this case both operands are already the same (larger) size, but the output from the convolution actually needs to be the size of the PSF. To accomplish this, use the `otf2psf` function from the `utils` library.



Important Notes:
----------------
1. You may use the utils module from class
2. All divisions must be safe (i.e. don't allow division by zero, most division here are scalar and really easy, one takes a bit more work)
3. Calculations should be conserved (it you need the Fourier transform of an image in two places, only take the Fourier transform once and save it in a variable)
4. The only loop is the loop over all of the iterations
5. Your variables do not need to line up with the math variables (however it may be helpful to do so)
6. The convolutions should be performed using Fourier transforms
7. The function should be self-contained (i.e. not use any global variables or global imports)
8. The constraints on the variables can be applied with a simple Numpy function we have used many times in class
9. Make sure to add documentation to your function

Testing
====
Load the cameraman image and blur it with a 7x7 Guassian kernel with $\sigma=10$. The convolution should use wrapping/circular padding. You can use the `guassian` function in the `utils` library to do this. Make sure the result is a unit8 image.

Using the `blind_deconvolve` function you created above on the image, giving it an all-ones 10x10 PSF, and the following number of iterations. View the result of the original image, the blurred image, and the blind-deconvolved image using 2, 5, 10, and 20 iterations. Next to each image plot the PSF (for the original image this is skipped, for the blurred image this is the Guassian kernel you used above, for the others this is the value returned by `blind_deconvolution`) in a figure with 6 rows and 2 columns making sure to add titles to each plot.

------
**How well did this first test do? What problems showed up? How did the found PSF compare to the actual PSF? How many iterations produced the best results?**

Testing 2
---------

Try this again but this time use constant-0 padding during the initial blur (which is more realistic if thinking about how the light is captured).

-----
**How do these results compare to the first testing results? What parts are better/worse? Explain why these differences occur even though the only thing changing is a small part of the intermediate blurred image.**

Testing 3
---------

Try this again but this time use the actual known PSF instead of all-ones PSF when deconvoling (and go back to the wrapping convolution).

-----
**How do these results compare to the first testing results? There are still some issues with the results though, where are they?**

Correction
=====

In the second testing section there was a problem with the more realistic constant padding. Find a simple (like one simplish line of code) solution to overcome this problem but still use constant padding. Write the code for this solution along with the display below.

-----
**Explain your solution and why it helps.**

Unblurred
=====

Run deconvolution on the original cameraman image (without blurring first) with a 15x15 all-ones PSF and use 60 iterations. Plot the resulting PSF (don't bother plotting the image).

-----
**Explain the resulting PSF**

Points
===

* 50 pts - correct `blind_deconvolution` implementation:
 * 10 pts - correct output produced in all cases
 * 10 pts - correctly implemented formulas
 * 8 pts - always keeping the PSF normalized so that it sums to 1 and applying the stability contraints
 * 6 pts - efficent (conserved calculations, single for loop, convolutions using Fourier transforms)
 * 5 pts - keeping data around for the appropriate amount of time
 * 5 pts - safe divisions
 * 3 pts - documentation
 * 3 pts - self-contained
* 20 pts - testing the results of the function and written answers
 * 10 pts - first testing and written answer
 * 5 pts - second testing and written answer
 * 5 pts - third testing and written answer
* 15 pts - correction
* 15 pts - unblurred